In [1]:
import os
import json
import numpy as np
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA

# Load embedding function
def load_embedding(input_embedding_name, model):
    if model.startswith('trans'):
        with open(input_embedding_name) as f:
            data = json.load(f)
        ent_embeddings = np.array(data['ent_embeddings.weight'])
        rel_embeddings = np.array(data['rel_embeddings.weight'])
        return ent_embeddings, rel_embeddings
    
    elif model == 'secureBERT':
        ent_embeddings = np.empty((0, 768), dtype=np.float32)
        for filename in sorted(os.listdir(input_embedding_name)):
            print(filename)

            if not filename.startswith('embeddings_chunk'):
                continue

            embedding = np.load(f'{input_embedding_name}/{filename}')

            print(ent_embeddings.shape, embedding.shape)

            ent_embeddings = np.concatenate((ent_embeddings, embedding), axis=0)
            print(filename, ent_embeddings.shape)

        print(f'Reducing entity embedding to ({DIM},)')
        print(ent_embeddings.shape, '->', end=' ')
        pca = PCA(n_components=DIM)
        ent_embeddings = pca.fit_transform(ent_embeddings)
        print(ent_embeddings.shape)

        rel_embeddings = np.load(f'{input_embedding_name}/relation.npy')
        print(f'Reducing relation embedding to ({len(rel_embeddings)},)')
        print(rel_embeddings.shape, '->', end=' ')
        pca = PCA(n_components=len(rel_embeddings))
        rel_embeddings = pca.fit_transform(rel_embeddings)
        print(rel_embeddings.shape)
        return ent_embeddings, rel_embeddings
    else:
        print('Error!!')
        return None
    


In [2]:
embedding_files = ["../data_new/source_data/embedding/secureBERT"]
model = 'secureBERT'
DIM = 250

# 输入文件列表
input_filenames = ["../data_new/graph/graph_without_benign.jsonl"]

for input_filename in tqdm(input_filenames):
    print("Start!")
    base, ext = os.path.splitext(input_filename)
    
    with open(input_filename, "r") as f:
        input_data = list(f)

    for embedding_file in tqdm(embedding_files):
        output_filename = f"{embedding_file.replace('.json', '_embedded').replace('.vec', '')}{ext}"
        print(f"output file name: {output_filename}")

        with open(output_filename, "w") as out_file:
            model = embedding_file.split('/')[-1].split('_')[0]
            ent_embeddings, rel_embeddings = load_embedding(embedding_file, model)
            # ...

            for line, data in tqdm(zip(input_data, input_data)):
                data = json.loads(data.strip())

                # Replace node_feat and edge_attr with embeddings
                data["node_feat"] = [ent_embeddings[node_id].tolist() if model == 'secureBERT' else ent_embeddings[node_id] for node_id in data["node_feat"]]
                data["edge_attr"] = [rel_embeddings[edge_id].tolist() for edge_id in data["edge_attr"]]

                # Convert the data back to a JSON string and write to the output file
                out_file.write(json.dumps(data) + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

Start!



  0%|          | 0/1 [00:00<?, ?it/s]

../data_new/source_data/embedding/secureBERT.jsonl
.ipynb_checkpoints
embeddings_chunk_0.npy
(0, 768) (160000, 768)
embeddings_chunk_0.npy (160000, 768)
embeddings_chunk_1.npy
(160000, 768) (160000, 768)
embeddings_chunk_1.npy (320000, 768)
embeddings_chunk_2.npy
(320000, 768) (160000, 768)
embeddings_chunk_2.npy (480000, 768)
embeddings_chunk_3.npy
(480000, 768) (160000, 768)
embeddings_chunk_3.npy (640000, 768)
embeddings_chunk_4.npy
(640000, 768) (160000, 768)
embeddings_chunk_4.npy (800000, 768)
embeddings_chunk_5.npy
(800000, 768) (160000, 768)
embeddings_chunk_5.npy (960000, 768)
embeddings_chunk_6.npy
(960000, 768) (160000, 768)
embeddings_chunk_6.npy (1120000, 768)
embeddings_chunk_7.npy
(1120000, 768) (51204, 768)
embeddings_chunk_7.npy (1171204, 768)
relation.npy
Reducing entity embedding to (250,)
(1171204, 768) -> (1171204, 250)
Reducing relation embedding to (26,)
(26, 768) -> (26, 26)




0it [00:00, ?it/s]

66it [00:00, 659.63it/s]

182it [00:00, 919.11it/s]

332it [00:00, 1177.87it/s]

467it [00:00, 1242.25it/s]

594it [00:00, 1250.73it/s]

720it [00:00, 1227.23it/s]

844it [00:00, 1227.94it/s]

972it [00:00, 1242.59it/s]

1117it [00:00, 1303.91it/s]

1248it [00:01, 1261.03it/s]

1378it [00:01, 1271.16it/s]

1506it [00:01, 1236.35it/s]

1632it [00:01, 1239.93it/s]

1757it [00:01, 1223.80it/s]

1899it [00:01, 1280.89it/s]

2028it [00:01, 1141.36it/s]

2146it [00:01, 762.22it/s] 

2241it [00:02, 631.10it/s]

2320it [00:02, 565.01it/s]

2388it [00:02, 525.29it/s]

2448it [00:02, 492.20it/s]

2502it [00:02, 477.16it/s]

2553it [00:02, 469.21it/s]

2602it [00:03, 462.03it/s]

2650it [00:03, 456.89it/s]

2697it [00:03, 427.22it/s]

2741it [00:03, 429.03it/s]

2785it [00:03, 422.32it/s]

2828it [00:03, 419.44it/s]

2871it [00:03, 418.19it/s]

2913it [00:03, 416.60it/s]

2956it [00:03, 416.16it/s]

2999it [00:04, 419.97it/s]

3139it [00:04, 704.22it/s]

3279it [00:04, 907.7

...

129483it [04:25, 10.50it/s]

129485it [04:25, 11.20it/s]

129487it [04:25, 11.53it/s]

129489it [04:25, 11.67it/s]

129491it [04:25, 12.03it/s]

129493it [04:25, 12.45it/s]

129495it [04:26, 12.51it/s]

129497it [04:26, 12.63it/s]

129499it [04:26, 12.79it/s]

129501it [04:26, 12.93it/s]

129503it [04:26, 12.73it/s]

129505it [04:26, 12.82it/s]

129507it [04:27, 12.56it/s]

129509it [04:27, 12.57it/s]

129511it [04:27, 12.78it/s]

129513it [04:27, 12.99it/s]

129515it [04:27, 12.90it/s]

129517it [04:27, 12.92it/s]

129519it [04:27, 13.16it/s]

129521it [04:28, 12.82it/s]

129523it [04:28, 12.21it/s]

129525it [04:28, 12.33it/s]

129527it [04:28, 12.46it/s]

129529it [04:28, 12.83it/s]

129531it [04:29, 11.27it/s]

129533it [04:29, 10.45it/s]

129535it [04:29, 10.96it/s]

129537it [04:29, 11.43it/s]

129539it [04:29, 11.84it/s]

129541it [04:29, 12.06it/s]

129543it [04:30, 12.00it/s]

129545it [04:30, 12.00it/s]

129547it [04:30, 11.88it/s]

129549it [04:30, 12.19it/s]

129551it [04:3

129947it [05:02,  9.43it/s]

129948it [05:02,  9.39it/s]

129949it [05:02,  9.37it/s]

129950it [05:03,  9.34it/s]

129951it [05:03,  9.32it/s]

129952it [05:03,  9.31it/s]

129953it [05:03,  9.30it/s]

129955it [05:03, 10.81it/s]

129957it [05:03, 11.52it/s]

129959it [05:03, 12.09it/s]

129961it [05:03, 12.23it/s]

129963it [05:04, 12.42it/s]

129965it [05:04, 12.64it/s]

129967it [05:04, 12.67it/s]

129969it [05:04, 12.82it/s]

129971it [05:04, 12.95it/s]

129973it [05:04, 12.79it/s]

129975it [05:05, 12.82it/s]

129977it [05:05, 12.49it/s]

129979it [05:05, 12.63it/s]

129981it [05:05, 12.90it/s]

129983it [05:05, 12.83it/s]

129985it [05:05, 13.05it/s]

129987it [05:05, 13.17it/s]

129989it [05:06, 13.01it/s]

129991it [05:06, 13.25it/s]

129993it [05:06, 13.20it/s]

129995it [05:06, 13.05it/s]

129997it [05:06, 13.09it/s]

129999it [05:06, 12.63it/s]

130006it [05:07, 24.81it/s]

130075it [05:07, 177.00it/s]

130152it [05:07, 322.47it/s]

130233it [05:07, 445.97it/s]

130316it [0

142598it [05:31, 606.95it/s]

142660it [05:31, 584.18it/s]

142720it [05:31, 550.45it/s]

142776it [05:31, 520.45it/s]

142829it [05:31, 508.09it/s]

142880it [05:31, 508.24it/s]

142935it [05:32, 518.84it/s]

142987it [05:32, 509.21it/s]

143038it [05:32, 495.80it/s]

143088it [05:32, 460.96it/s]

143135it [05:32, 451.46it/s]

143181it [05:32, 442.01it/s]

143226it [05:32, 439.92it/s]

143271it [05:32, 430.91it/s]

143315it [05:32, 426.44it/s]

143359it [05:32, 428.16it/s]

143402it [05:33, 419.40it/s]

143444it [05:33, 392.70it/s]

143484it [05:33, 385.19it/s]

143523it [05:33, 381.44it/s]

143562it [05:33, 377.06it/s]

143600it [05:33, 374.99it/s]

143638it [05:33, 353.70it/s]

143677it [05:33, 363.08it/s]

143715it [05:33, 359.57it/s]

143758it [05:34, 378.27it/s]

143797it [05:34, 374.32it/s]

143837it [05:34, 378.58it/s]

143875it [05:34, 359.02it/s]

143912it [05:34, 342.94it/s]

143947it [05:34, 332.40it/s]

143981it [05:34, 332.46it/s]

144038it [05:34, 398.39it/s]

144106it [

152100it [06:00, 268.33it/s]

152157it [06:00, 349.72it/s]

152218it [06:01, 423.86it/s]

152279it [06:01, 467.52it/s]

152338it [06:01, 500.58it/s]

152399it [06:01, 528.71it/s]

152461it [06:01, 548.97it/s]

152524it [06:01, 570.53it/s]

152587it [06:01, 585.94it/s]

152648it [06:01, 589.18it/s]

152708it [06:01, 581.01it/s]

152767it [06:02, 578.60it/s]

152830it [06:02, 591.02it/s]

152891it [06:02, 596.11it/s]

152951it [06:02, 583.16it/s]

153010it [06:02, 488.45it/s]

153062it [06:02, 295.40it/s]

153103it [06:03, 233.74it/s]

153136it [06:03, 204.43it/s]

153163it [06:03, 187.07it/s]

153186it [06:03, 175.26it/s]

153207it [06:03, 166.10it/s]

153226it [06:04, 158.21it/s]

153243it [06:04, 152.86it/s]

153259it [06:04, 145.08it/s]

153274it [06:04, 143.64it/s]

153289it [06:04, 141.37it/s]

153304it [06:04, 138.85it/s]

153318it [06:04, 136.71it/s]

153332it [06:04, 134.34it/s]

153346it [06:04, 129.76it/s]

153359it [06:05, 126.38it/s]

153372it [06:05, 124.39it/s]

153385it [

158972it [06:30, 65.05it/s]

158979it [06:31, 64.11it/s]

158986it [06:31, 62.94it/s]

158993it [06:31, 61.95it/s]

159000it [06:31, 60.18it/s]

159151it [06:31, 449.97it/s]

159304it [06:31, 748.24it/s]

159450it [06:31, 949.06it/s]

159607it [06:31, 1126.69it/s]

159762it [06:31, 1249.38it/s]

159917it [06:31, 1335.33it/s]

160053it [06:32, 1296.63it/s]

160185it [06:32, 1234.97it/s]

160321it [06:32, 1268.41it/s]

160471it [06:32, 1333.51it/s]

160606it [06:32, 1319.35it/s]

160748it [06:32, 1348.09it/s]

160902it [06:32, 1403.57it/s]

161047it [06:32, 1414.86it/s]

161189it [06:32, 1407.75it/s]

161331it [06:33, 1366.24it/s]

161469it [06:33, 1351.84it/s]

161605it [06:33, 1344.61it/s]

161740it [06:33, 1336.37it/s]

161896it [06:33, 1401.55it/s]

162037it [06:33, 723.60it/s] 

162146it [06:34, 297.34it/s]

162226it [06:35, 208.76it/s]

162285it [06:36, 178.15it/s]

162329it [06:36, 153.18it/s]

162363it [06:37, 135.15it/s]

162389it [06:37, 129.08it/s]

162411it [06:37, 122.75it/s